In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [3]:
# First, read the data into a pandas data frame
json_df = pd.read_json('data/world_bank_projects.json')
#json_df.info()

# Find the 10 countries with most projects
Pandas has built in function (*value_counts*) to count number of occurances of a value in a column. 
Here, simply count the times a country name appears

In [4]:
# Pandas has built in function (value_counts) to count number of occurances of 
json_df.countryshortname.value_counts()[:10]

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Morocco               12
Bangladesh            12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

# Find the top 10 major project themes (using column 'mjtheme_namecode')
The __mjtheme_namecode__ codes and corresponding descriptions are contained in a list if we simply use the read_json function above. Instead, I first load the json file, and then use json_normalize to expand into one row for each code for each project.

In [5]:

data = json.load(open('data/world_bank_projects.json'))
# Use the json_normalize as above. More columns can be read in if needed
expanded_data = json_normalize(data, 'mjtheme_namecode', ['_id','countryshortname', 'project_name'])
#expanded_data.info()

Then, as above, we can use value_counts to find the most common project themes

In [6]:
expanded_data.code.value_counts()[:10]

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

# In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
To first find the name for each code, I group by code, and then take the max of the names. Here, max() works, since having a description will be 'greater than' an empty string

In [7]:
code_names = expanded_data[['code', 'name']].groupby('code').max()
#code_names

Now that I have a dataframe with each code and codename, I can merge this (outer join) with the original data frame that has the missing data.

In [8]:
filled_missing = pd.merge(expanded_data, code_names, how='outer', left_on='code', right_index=True) 
# Create new column that is the max of the two description strings
filled_missing['name'] = filled_missing[['name_x','name_y']].apply(max, axis=1)
filled_missing = filled_missing.drop(columns=['name_x','name_y'])

In [9]:
filled_missing
#filled_missing.info()

,code,_id,countryshortname,project_name,name
0,8,{'$oid': '52b213b38594d8a2be17c780'},Ethiopia,Ethiopia General Education Quality Improvement...,Human development
17,8,{'$oid': '52b213b38594d8a2be17c787'},China,China Renewable Energy Scale-Up Program Phase II,Human development
41,8,{'$oid': '52b213b38594d8a2be17c791'},Madagascar,Emergency Support to Education For all Project,Human development
43,8,{'$oid': '52b213b38594d8a2be17c792'},Cambodia,Additional Financing for the Second Health Sec...,Human development
44,8,{'$oid': '52b213b38594d8a2be17c792'},Cambodia,Additional Financing for the Second Health Sec...,Human development
47,8,{'$oid': '52b213b38594d8a2be17c792'},Cambodia,Additional Financing for the Second Health Sec...,Human development
57,8,{'$oid': '52b213b38594d8a2be17c797'},Tajikistan,Tajikistan JSDF Nutrition Grant Scale Up,Human development
58,8,{'$oid': '52b213b38594d8a2be17c797'},Tajikistan,Tajikistan JSDF Nutrition Grant Scale Up,Human development
69,8,{'$oid': '52b213b38594d8a2be17c79a'},Lao People's Democratic Republic,Lao PDR Ninth Poverty Reduction Support Operation,Human development
79,8,{'$oid': '52b213b38594d8a2be17c79e'},Angola,Angola Learning for All Project,Human development
